In [70]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [71]:
%mkdir ch6
!touch ch6/__init__.py

mkdir: cannot create directory ‘ch6’: File exists


In [0]:
import imp

## 6.1 案例分析： 重构策略模式

### 6.1.1 经典的“策略”模式

- 使用“策略”设计模式处理订单折扣的 UML 类图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200330104832.png width=800>

- 《设计模式：可复用面向对象软件的基础》一书是这样概述“策略”模式的
  - 定义一系列算法，把它们一一封装起来，并且使它们可以相互替换
  - 本模式使得算法可以独立于使用它的客户而变化

#### 示例： 电商折扣计算

###### 相关规则

- 一个网店制定了下述折扣规则:
  - 有 1000 或以上积分的顾客，每个订单享 5% 折扣
  - 同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
  - 订单中的不同商品达到 10 个或以上，享 7% 折扣
- 简单起见，假定一个订单一次只能享用一个折扣

###### “策略”模式的 UML 类图见图 6-1，其中涉及下列内容

- 上下文
  - 把一些计算委托给实现不同算法的可互换组件，它提供服务。在这个电商示例中，上下文是 Order，它会根据不同的算法计算促销折扣
- 策略
  - 实现不同算法的组件共同的接口。在这个示例中，名为 `Promotion` 的抽象类扮演这个角色
- 具体策略
  - “策略”的具体子类。`fidelityPromo`、`BulkPromo` 和 `LargeOrderPromo` 是这里实现的三个具体策略

###### 示例 6-1　实现 Order 类，支持插入式折扣策略

In [73]:
%%writefile ch6/order.py
"""示例 6-1"""


from abc import ABC, abstractmethod
from collections import namedtuple


Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity


class Order:  # 上下文
  def __init__(self, customer, cart, promotation=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotation = promotation

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotation is None:
      discount = 0
    else:
      discount = self.promotation.discount(self)
    return self.total() - discount
  
  def __repr__(self):
    fmt = '<Order total: {:.2f} due:{:.2f}>'
    return fmt.format(self.total(), self.due())


class Promotation(ABC):  # 策略：抽象基类
  @abstractmethod
  def discount(self, order):
    """返回折扣金额（正值）"""
    pass


class FidelityPromo(Promotation):  # 第一个具体策略
  """为积分为 1000 或以上的顾客提供 5% 的折扣"""
  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity > 1000 else 0


class BulkItemPromo(Promotation):  # 第二个策略
  """单个商品为 20 个或以上时，提供 10% 折扣"""
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotation):  # 第三个策略
  """订单中的不同商品达到10个或以上时提供7%折扣"""
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    return order.total() * 0.07 if len(distinct_items) >= 10 else 0

Overwriting ch6/order.py


###### 示例 6-2　使用不同促销折扣的 Order 类示例

In [0]:
import ch6.order
imp.reload(ch6.order)
from ch6.order import Customer, LineItem, Order, FidelityPromo, BulkItemPromo, LargeOrderPromo

In [0]:
joe = Customer('John Doe', 0) 
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), 
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)]

- 按用户积分折扣

In [76]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due:42.00>

In [77]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due:39.90>

- 按订单中单个的商品数量折扣

In [78]:
banana_cart = [LineItem('banana', 30, .5),
        LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due:28.50>

- 按订单中的商品种类数折扣

In [79]:
large_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, large_order, LargeOrderPromo())

<Order total: 10.00 due:9.30>

In [80]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due:42.00>

### 6.1.1 使用函数实现策略模式

- 在示例 6-1 中，每个具体策略都是一个类，而且都只定义了一个方法，即 discount。此外，策略实例没有状态（没有实例属性）
  - 这些类与普通的函数基本相同
  - 示例 6-3 是对示例 6-1的重构，把具体策略换成了简单的函数，而且去掉了 Promo 抽象类。

###### 示例 6-3　Order 类和使用函数实现的折扣策略

In [81]:
%%writefile ch6/order_func.py
"""示例 6-1"""


from abc import ABC, abstractmethod
from collections import namedtuple


Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity


class Order:  # 上下文
  def __init__(self, customer, cart, promotation=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotation = promotation

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotation is None:
      discount = 0
    else:
      discount = self.promotation(self)
    return self.total() - discount
  
  def __repr__(self):
    fmt = '<Order total: {:.2f} due:{:.2f}>'
    return fmt.format(self.total(), self.due())


def fidelity_promo(order):  # 第一个具体策略
  """为积分为 1000 或以上的顾客提供 5% 的折扣"""
  return order.total() * 0.05 if order.customer.fidelity > 1000 else 0


def bulk_item_promo(order):  # 第二个策略
  """单个商品为 20 个或以上时，提供 10% 折扣"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount


def large_order_promo(order):  # 第三个策略
  """订单中的不同商品达到10个或以上时提供7%折扣"""
  distinct_items = {item.product for item in order.cart}
  return order.total() * 0.07 if len(distinct_items) >= 10 else 0

Overwriting ch6/order_func.py


###### 示例 6-4　使用函数实现的促销折扣的 Order 类示例

In [0]:
import ch6.order_func
imp.reload(ch6.order_func)
from ch6.order_func import Customer, LineItem, Order, fidelity_promo, bulk_item_promo, large_order_promo

In [0]:
joe = Customer('John Doe', 0) 
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), 
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)]

- 按用户积分折扣

In [84]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due:42.00>

In [85]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due:39.90>

- 按订单中单个的商品数量折扣

In [86]:
banana_cart = [LineItem('banana', 30, .5),
        LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due:28.50>

- 按订单中的商品种类数折扣

In [87]:
large_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, large_order, large_order_promo)

<Order total: 10.00 due:9.30>

In [88]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due:42.00>

### 6.1.3 选择最佳策略：简单的方式

###### 示例 6-6　`best_promo` 迭代一个函数列表，并找出折扣额度最大的

In [89]:
%%writefile -a ch6/order_func.py



promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order):
  """选择可用的最佳策略
  """
  return max(promo(order) for promo in promos)

Appending to ch6/order_func.py


###### 示例 6-5　best_promo 函数计算所有折扣，并返回额度最大的

In [0]:
imp.reload(ch6.order_func)
from ch6.order_func import best_promo

In [91]:
Order(joe, large_order, best_promo)

<Order total: 10.00 due:9.30>

In [92]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due:28.50>

In [93]:
Order(ann, cart, best_promo)

<Order total: 42.00 due:39.90>

### 6.1.4 找出模块中的全部策略

- 6.1.3 中如果想添加新的促销策略，要定义相应的函数，还要将其它添加到 `promos` 列表中
  - 否则，当新促销函数显式地作为参数传给 `Order` 时，它是可用的，但是 `best_promo` 不会考虑它

#### `globals()`

- 返回一个字典，表示当前的全局符号表
  - 这个符号表始终针对当前模块（对函数或方法来说，是指定义它们的模块，而不是调用它们的模块）

###### 示例 6-7　内省模块的全局命名空间，构建 `promos` 列表

In [94]:
%%writefile -a ch6/order_func.py



promos_auto = [globals()[name] for name in globals()
        if name.endswith('_promo')
        and name != 'best_promo']


def best_promo_auto(order):
  """例6-7 使用自动遍历所得的列表"""
  return max(promo(order) for promo in promos_auto)

Appending to ch6/order_func.py


In [0]:
imp.reload(ch6.order_func)
from ch6.order_func import best_promo_auto

In [96]:
Order(joe, large_order, best_promo_auto)

<Order total: 10.00 due:9.30>

In [97]:
Order(joe, banana_cart, best_promo_auto)

<Order total: 30.00 due:28.50>

In [98]:
Order(ann, cart, best_promo_auto)

<Order total: 42.00 due:39.90>

###### 示例 6-8　内省单独的 promotions 模块，构建 promos 列表

- 只含有策略函数的 `promotions` 模块

In [107]:
%%writefile ch6/promotions.py
def fidelity_promo(order):  # 第一个具体策略
  """为积分为 1000 或以上的顾客提供 5% 的折扣"""
  return order.total() * 0.05 if order.customer.fidelity > 1000 else 0


def bulk_item_promo(order):  # 第二个策略
  """单个商品为 20 个或以上时，提供 10% 折扣"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount


def large_order_promo(order):  # 第三个策略
  """订单中的不同商品达到10个或以上时提供7%折扣"""
  distinct_items = {item.product for item in order.cart}
  return order.total() * 0.07 if len(distinct_items) >= 10 else 0

Overwriting ch6/promotions.py


- 除了 `best_promo` 不含有其它策略的模块

In [114]:
%%writefile ch6/order_without_promos.py
"""示例 6-1"""


from abc import ABC, abstractmethod
from collections import namedtuple
import ch6.promotions as promotions
import inspect


Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity


class Order:  # 上下文
  def __init__(self, customer, cart, promotation=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotation = promotation

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotation is None:
      discount = 0
    else:
      discount = self.promotation(self)
    return self.total() - discount
  
  def __repr__(self):
    fmt = '<Order total: {:.2f} due:{:.2f}>'
    return fmt.format(self.total(), self.due())


promos = [func for name, func in 
          inspect.getmembers(promotions, inspect.isfunction)]


def best_promo(order):
  """选择是佳的折扣策略
  """
  return max(promo(order) for promo in promos)

Overwriting ch6/order_without_promos.py


In [0]:
import ch6.order_without_promos
imp.reload(ch6.order_without_promos)
from ch6.order_without_promos import best_promo

In [117]:
Order(joe, large_order, best_promo)

<Order total: 10.00 due:9.30>

In [118]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due:28.50>

In [119]:
Order(ann, cart, best_promo)

<Order total: 42.00 due:39.90>

## 6.2 命令模式

- “命令”模式也可以通过将函数作为参数传递而简化

- 菜单驱动的文本编辑器的 UML 类图，使用“命令”设计模实现
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200330133137.png width=800>
  - 各个命令可以有不同的接收者（实现操作的对象）
  - 对PasteCommand 来说，接收者是 Document。对 OpenCommand 来说，接收者是应用程序

- “命令”模式的目的是解耦调用操作的对象（调用者）和提供实现的对象（接收者）
  - 在《设计模式：可复用面向对象软件的基础》所举的示例中，调用者是图形应用程序中的菜单项，而接收者是被编辑的文档或应用程序自身。
- 这个模式的做法是，在二者之间放一个 Command 对象，让它实现只有一个方法（execute）的接口，调用接收者中的方法执行所需的操作
  - 调用者无需了解接收者的接口
  - 不同的接收者可以适应不同的 Command 子类
  - 调用者有一个具体的命令，通过调用 execute 方法执行
    - 注意，图 6-2 中的 MacroCommand 可能保存一系列命令，它的execute() 方法会在各个命令上调用相同的方法。

- 可以不为调用者提供一个 Command 实例，而是给它一个函数
  - 此时，调用者不用调用 `command.execute()`，直接调用 `command()` 即可
  - `MacroCommand` 可以实现成定义了 `__call__` 方法的类
    - 这样，MacroCommand 的实例就是可调用对象，各自维护着一个函数列表，供以后调用

###### 示例 6-9　MacroCommand 的各个实例都在内部存储着命令列表

In [0]:
class MicroCommand:
  """一个执行一组命令的命令"""
  def __init__(self, commands):
    self.commands = list(commnads)  # 使用 commands 参数构建一个列表，这样能确保参数是可迭代对象，还能在各个 MacroCommand 实例中保存各个命令引用的副本
  
  def __call__(self):
    for command in self.commands:  # 调用 MacroCommand 实例时，self.commands 中的各个命令依序执行
      command()